In [2]:
#import GetOldTweets3 as got
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv("Datasets/potholeData.csv")

In [4]:
csv_input = pd.read_csv('Datasets/potholeData.csv')
l = []
for i in range (0, 100):
    l.append("Pothole")
    i += 1
#print(l)
csv_input['Category'] = l
csv_input.to_csv('Datasets/potholeData.csv', index=False)   #this writes back to your file


In [5]:
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Adithi
[nltk_data]     Satish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.svm import SVC
import pickle

dfcat = pd.read_csv("Datasets/cat_dataset.csv")

def preprocess(df,col):
    stemmer = PorterStemmer()
    words = stopwords.words("english")

    #the preprocessing is done in 4 steps:

    #removing punctuation
    #removing stopwords like 'the', 'this','as',etc
    #conversion of the entire text to lower case
    #Stemming: reducing the number of inflectional forms of words by reducing all to their common stem.
    #For example, 'argue','arguing','argued' are all reduced to 'argu'
    df[col] = df[col].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

    return df

def train_SVC(train_tfIdf, y_train):
    #building text classification model using Linear Kernel SVC Classifier (has highest accuracy)
    classifier = SVC(kernel='linear') #accuracy obtained for linear kernel = 83.28%
    classifier.fit(train_tfIdf, y_train) #fitting the classifier onto the training data
    filename = "linearkernelSVC.sav"
    pickle.dump(classifier,open(filename,"wb"))

In [7]:
def addCat(df, csv):
    desc = list(dfcat["description"]) + list(df.Tweet)
    categories = list(dfcat["category"]) + [''] * len(df.Tweet)

    complaints = pd.DataFrame()
    complaints["description"] = desc
    complaints["category"] = categories
    complaints.head(10)

    complaints = preprocess(complaints,"description")
    trainX = complaints.description[:10723]
    trainY = complaints.category[:10723]
    testX = complaints.description[10723:]

    vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7) #initializing the vector
    train_tfIdf = vectorizer_tfidf.fit_transform(trainX.values.astype('U')) #astype('U') converts the dataframe into a Unicode array
    test_tfIdf = vectorizer_tfidf.transform(testX.values.astype('U'))#transforming the text into frequency vectors

    train_SVC(train_tfIdf,trainY)
    classifier = pickle.load(open("linearkernelSVC.sav","rb"))
    pred = classifier.predict(test_tfIdf) #predictions made on the unseen data

    df["category"] = pred
    df.head(10)
    df.to_csv("Datasets/" + csv)

#addCat(dfbbmp,"BBMPData1.csv")

## Adding categories for BBMP Data

In [8]:
dfbbmp = pd.read_csv("Datasets/BBMPData.csv")
dfbbmp

,Unnamed: 0,Tweet,Date,Username,Hashtags
0,0,@BBMP_MAYOR @BBMPCOMM Sarjapur road from Decat...,2020-01-31 17:16:38+00:00,shroff_bhavesh,NaN
1,1,The future is now. Get ready for #GalaxyZFold2...,2020-09-01 15:45:00+00:00,SamsungIndia,#GalaxyZFold2 #Samsung
2,2,"Not only this, the burning of garbage and wast...",2020-01-31 17:12:40+00:00,Sudhindrakul1,NaN
3,3,@Tejasvi_Surya @CMofKarnataka @narendramodi Mr...,2020-01-31 17:11:38+00:00,shroff_bhavesh,NaN
4,4,But residents are equally responsible here. Re...,2020-01-31 13:23:51+00:00,Maheshkumar891,NaN
...,...,...,...,...,...
2995,2995,This is a fairly populated area and there are ...,2019-08-17 13:36:38+00:00,divya_2377,NaN
2996,2996,The pictures that I posted are of Gottigere-Ke...,2019-08-17 13:31:38+00:00,divya_2377,NaN
2997,2997,Who will be responsible for this? #BBMP Next t...,2019-08-17 13:30:43+00:00,be1cf20811c44cd,#BBMP
2998,2998,Am glad Shivani you replied This is ward no 19...,2019-08-17 13:23:50+00:00,divya_2377,NaN


## Adding Categories for BWSSB

In [14]:
dfbwssb = pd.read_csv("Datasets/BWSSBData.csv")
dfbwssb

,Unnamed: 0,Tweet,Date,Username,Hashtags
0,0,You can't clean Bangalore as there's no proper...,2020-01-31 17:25:11+00:00,mahesh3028,NaN
1,1,The future is now. Get ready for #GalaxyZFold2...,2020-09-01 15:45:00+00:00,SamsungIndia,#GalaxyZFold2 #Samsung
2,2,@bwssbchairman @chairmanbwssb @BBMPCOMM @PCMoh...,2020-01-31 17:04:19+00:00,rajmalhotrasid,#Bengaluru #bwssb #BBMP
3,3,"Dy cm sir, basic steps- till metro stop new of...",2020-01-31 16:17:24+00:00,Viswana70378552,NaN
4,4,#BWSSB has developed patented tech for self -r...,2020-01-31 16:17:06+00:00,SelfDoubtist,#BWSSB
...,...,...,...,...,...
1995,1995,"BWSSB contribution to yellenahalli,begur road ...",2019-09-25 03:20:51+00:00,arolitriveni,NaN
1996,1996,No one is answerable to citizens is it? @BBMPC...,2019-09-25 03:12:53+00:00,dmoneymachine,NaN
1997,1997,Where has all the money from various taxes gon...,2019-09-25 03:08:19+00:00,makingindiawork,NaN
1998,1998,Pathetic road conditions due to Bwssb work sin...,2019-09-25 03:04:36+00:00,manishbucha82,#BBMP


In [11]:
#addCat(dfbwssb,"BWSSBData1.csv")

## Adding Categories for Traffic/Parking Data

In [15]:
dftraf = pd.read_csv("Datasets/trafficDataset.csv")
dftraf

,Unnamed: 0,Tweet,Date,Username,Hashtags
0,0,@BMTC_BENGALURU thinks that all roads are belo...,2020-01-31 14:22:48+00:00,NaveenDamle,NaN
1,1,The future is now. Get ready for #GalaxyZFold2...,2020-09-01 15:45:00+00:00,SamsungIndia,#GalaxyZFold2 #Samsung
2,2,This road rage has become so common these days...,2020-01-31 09:29:00+00:00,travelofindia,NaN
3,3,These BMTC bus drivers don't follow basic road...,2020-01-31 08:38:03+00:00,cishan,NaN
4,4,And Electronic City and ORR and Sarjapur Road....,2020-01-31 06:28:10+00:00,srinualavilli,NaN
...,...,...,...,...,...
995,995,Things raindrops hurtling toward Bangalore see...,2019-08-17 03:27:43+00:00,rainbotblr,#bangalorerains
996,996,who will look After traffic issues happening w...,2019-08-17 01:26:59+00:00,rakeshsahoo318,NaN
997,997,@bjparvind @BSYBJP @Tejasvi_Surya at Iblur bus...,2019-08-16 19:32:58+00:00,gyan_,NaN
998,998,@nitin_gadkari @MORTHIndia @tdkarnataka Sir we...,2019-08-16 17:17:26+00:00,aa10860303424b1,NaN


In [16]:
#addCat(dftraf,"trafficData1.csv")